# Cleaning and Merging Players Data for Playing Minutes Overview 
## General Information  
**Objective**  
This script cleans and merges scraped data located in the data/raw folder to allow in-depth analysis of player statistics.

**Scope**
This script processes and cleans the following datasets:

- players_current.csv → cleaned to players_cleaned.csv
- lineups.csv → cleaned and saved as lineups_prep.csv
- lineups_prep.csv and match_events.csv → merged and saved as lineups_cleaned.csv
- Using lineups_prep.csv to calculate playing minutes for each player → saved as player_stats_cleaned.csv

**Methodology**  

- Clean datasets for proper format and calculations.
- Merge lineup and match events data.
- Calculate individual player statistics.

**Usage**  
Cleaned datasets will be saved to the data folder for further analysis.

In [ ]:
# Core Python Libraries
import os
import re
import time
from datetime import datetime

# Data Manipulation Libraries
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder


## 1. Cleaning Players Data  
**Description**  
This section cleans the players_current.csv file by:

- Converting market value formats.
- Parsing birthdates into a standardized format.
- Calculating player ages.

In [ ]:
# Function to clean and convert the market values
def clean_market_value(value_str):
    """
    Cleans and standardizes market value strings.
    Converts "Mio." to millions and "Tsd." to thousands.
    """
    value_str = (
        value_str.replace('€', '')  # Remove the euro symbol
        .replace('â‚¬', '')  # Remove any encoding artifacts
        .replace('Tsd.', 'k')  # Replace "Tsd." with "k" for consistency
        .replace('Mio.', 'm')  # Replace "Mio." with "m" for consistency
        .replace(',', '.')  # Replace commas with dots for float conversion
        .strip()  # Remove leading and trailing spaces
    )
    market_value = 0.0
    if 'm' in value_str:
        # Convert values in millions
        market_value = float(value_str.replace('m', ''))
    elif 'k' in value_str:
        # Convert values in thousands (divided by 1000 to get millions)
        market_value = float(value_str.replace('k', '')) / 1000
    return market_value

# Function to extract birthdate from the Birthdate column
def extract_birthdate(birthdate_str):
    """
    Extracts the birthdate part from a string formatted as 'Date (Age)'.
    """
    birthdate_part, _ = birthdate_str.split('(')  # Split the string at '(' and keep the date part
    return birthdate_part.strip()

# Function to parse date with error handling
def parse_date(date_str, format='%d.%m.%Y'):
    """
    Converts a date string into a datetime object.
    Returns NaT if parsing fails.
    """
    try:
        return pd.to_datetime(date_str, format=format, errors='coerce')  # Handle errors gracefully
    except ValueError:
        return pd.NaT  # Return NaT for invalid dates

# Calculate age based on birthdate
def calculate_age(birthdate):
    """
    Calculates the age of a player based on their birthdate.
    """
    if pd.notnull(birthdate):
        today = datetime.today()
        # Calculate age, adjusting for whether the birthdate has occurred this year
        age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
        return age
    return None

# Path to the raw data file
file_path = '../data/raw/players_NT_2024-11-12.csv'  # Replace with the actual file path

# Load the data
df = pd.read_csv(file_path)

# Apply the function to clean and convert market values
df['Value'] = df['Value'].apply(clean_market_value)

# Extract and parse the birthdate
df['Birthdate'] = (
    df['Birthdate']
    .apply(extract_birthdate)  # Extract the birthdate
    .apply(lambda x: parse_date(x, format='%d.%m.%Y'))  # Convert the string to a datetime object
)

# Calculate age based on the birthdate
df['Age'] = df['Birthdate'].apply(calculate_age)

# Remove any unnecessary spaces in the 'Name' column
df['Name'] = df['Name'].str.strip()

# Save the modified DataFrame to a new CSV file
df.to_csv('../data/cleaned/players_cleaned.csv', index=False)  # Replace with preferred name

# Display the first few rows of the cleaned DataFrame
df.head(10)


## 2. Creating Unique Players Dataset  

Generates a unique dataset of players with clean birthdate formats and calculated ages.

In [ ]:
# Load the cleaned players data file
file_path = '../data/cleaned/players_cleaned.csv'
df = pd.read_csv(file_path)

# Keep only 'Name' and 'Birthdate' columns for further processing
df = df[['Name', 'Birthdate']]

# Remove any unnecessary spaces from the 'Name' column
df['Name'] = df['Name'].str.strip()

# Function to parse date with the correct format '%Y-%m-%d'
def parse_date_correct(date_str):
    """
    Parses date strings into datetime objects using the specified format.
    Returns NaT for invalid or missing values.
    """
    try:
        # Parse the date using the expected format
        parsed_date = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
        return parsed_date
    except ValueError:
        # Return NaT if parsing fails
        return pd.NaT

# Apply the corrected date parsing function to the 'Birthdate' column
df['Birthdate'] = df['Birthdate'].apply(parse_date_correct)

# Function to calculate age based on birthdate
def calculate_age(birthdate):
    """
    Calculates the age of a player based on their birthdate.
    """
    if pd.notnull(birthdate):
        today = datetime.today()
        # Compute age, accounting for whether the birthdate has occurred this year
        age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
        return age
    return None

# Calculate the current age and add it as a new column 'Age'
df['Age'] = df['Birthdate'].apply(calculate_age)

# Drop duplicate rows based on 'Name' and 'Birthdate'
df = df.drop_duplicates()

# Save the cleaned and processed data to a new CSV file
output_file_path = '../data/cleaned/players_unique.csv'
df.to_csv(output_file_path, index=False)

print("File saved successfully at:", output_file_path)

# Display the first few rows of the cleaned DataFrame for verification
print(df.head(10))


## 3. Merging National Team Stats  
 
This section merges player national team statistics with cleaned player data to provide additional insights.

In [ ]:

# Load the cleaned detailed stats file and the national team stats file
file1_path = '../data/raw/players_detailed_stats_NT.csv'  # Adjust path if necessary
file2_path = '../data/raw/players_NT_stats_incl_ID.csv'   # Adjust path if necessary

# Read the CSV files into DataFrames
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Merge the two DataFrames on "Player ID" and "National Team ID"
merged_df = pd.merge(
    df1,
    df2[['Player ID', 'National Team', 'Debut Date', 'National Team ID']],  # Include only relevant columns
    on=['Player ID', 'National Team ID'],  # Merge on these common columns
    how='left'  # Keep all rows from df1 and add matching rows from df2
)

# Rename "National Team" column to "Club" to standardize terminology
merged_df = merged_df.rename(columns={'National Team': 'Club'})

# Convert "Debut Date" column to a datetime object with the expected day-month-year format
merged_df['Debut Date'] = pd.to_datetime(
    merged_df['Debut Date'], format='%d.%m.%Y', errors='coerce'
)

# Determine "Season Start" and "Season End" based on the "Debut Date" month
merged_df['Season Start'] = merged_df['Debut Date'].apply(
    lambda x: x.year if pd.notnull(x) and x.month >= 8 else None  # Start in current year if the debut was in or after August
)
merged_df['Season End'] = merged_df['Debut Date'].apply(
    lambda x: x.year if pd.notnull(x) and x.month <= 7 else None  # End in current year if the debut was in or before July
)

# Save the merged DataFrame to a new CSV file
output_path = '../data/cleaned/players_merged_NT_stats.csv'  # Specify the output path
merged_df.to_csv(output_path, index=False)

# Display the first few rows of the merged DataFrame for verification
print(merged_df.head(10))

# Confirm that the merged file has been saved successfully
print("Merged data with 'Season Start' and 'Season End' columns saved to:", output_path)


In [ ]:
# File paths for data
file_path = '../data/cleaned/players_merged_NT_stats.csv'  # Path to the merged dataset
players_file_path = '../data/cleaned/players_incl_ID.csv'  # Path to the players dataset with birthdate information

# Load the merged dataset and the players dataset
merged_df = pd.read_csv(file_path)
players_df = pd.read_csv(players_file_path)

# Merge the "Birthdate" column from players dataset into the merged dataset
merged_df = pd.merge(
    merged_df, 
    players_df[['Player ID', 'Birthdate']], 
    on='Player ID', 
    how='left'  # Ensure all rows in merged_df are retained
)

# Convert the "Birthdate" column to a datetime format
merged_df['Birthdate'] = pd.to_datetime(
    merged_df['Birthdate'], format='%d/%m/%Y', errors='coerce'
)

# Define a function to calculate missing "Debut Date" based on "Club" and "Birthdate"
def calculate_debut_date(row):
    if pd.notnull(row['Debut Date']):  # If "Debut Date" is already present, retain it
        return row['Debut Date']
    elif pd.notnull(row['Birthdate']) and pd.notnull(row['Club']):
        # Check for youth categories in "Club" and calculate the approximate debut date
        for category, age_offset in [
            ('U15', 14), ('U16', 15), ('U17', 16), ('U18', 17), 
            ('U19', 18), ('U20', 19), ('U21', 20), ('U22', 21), ('U23', 22)
        ]:
            if category in row['Club']:
                # Add the offset in years to the birthdate to estimate debut date
                return row['Birthdate'] + timedelta(days=age_offset * 365.25)
    return None  # Return None if no conditions are met

# Apply the function to calculate missing "Debut Dates"
merged_df['Debut Date'] = merged_df.apply(calculate_debut_date, axis=1)

# Format "Debut Date" column consistently as 'DD/MM/YYYY'
merged_df['Debut Date'] = pd.to_datetime(
    merged_df['Debut Date']
).dt.strftime('%d/%m/%Y')

# Display the first 20 rows of relevant columns for verification
print(merged_df[['Player ID', 'Birthdate', 'Club', 'Debut Date']].head(20))

# Save the updated DataFrame to a CSV file
output_path = '../data/cleaned/players_merged_NT_stats.csv'
merged_df.to_csv(output_path, index=False)

# Confirm that the file has been saved
print("Updated data with calculated 'Debut Date' saved to:", output_path)


## 4. Adding Season Information  
Description
This section enriches data with season start and end details where missing values exist.

In [ ]:
# File path for the cleaned and merged dataset with Season Start and Season End columns
file_path = '../data/cleaned/players_merged_NT_stats.csv'

# Load the dataset
merged_df = pd.read_csv(file_path)

# Fill missing "Season End" where "Season Start" is filled
merged_df['Season End'] = merged_df.apply(
    lambda row: row['Season Start'] + 1 
    if pd.notnull(row['Season Start']) and pd.isnull(row['Season End']) 
    else row['Season End'], 
    axis=1
)

# Fill missing "Season Start" where "Season End" is filled
merged_df['Season Start'] = merged_df.apply(
    lambda row: row['Season End'] - 1 
    if pd.notnull(row['Season End']) and pd.isnull(row['Season Start']) 
    else row['Season Start'], 
    axis=1
)

# Save the updated DataFrame to a new CSV file
output_path = '../data/cleaned/players_merged_NT_stats.csv'
merged_df.to_csv(output_path, index=False)

# Display the first few rows of the updated DataFrame for verification
print(merged_df.head(10))

# Confirm that the updated file has been saved
print("Data with adjusted 'Season Start' and 'Season End' saved to:", output_path)


## 5. Adding Competition to Identify National Teams  

This section processes the dataset to add competition information, clean empty fields, and remove irrelevant rows.

In [ ]:
# Load the updated file containing merged national team statistics
file_path = '../data/cleaned/players_merged_NT_stats.csv'
merged_df = pd.read_csv(file_path)

# Step 1: Remove rows where the "Season" column is blank
merged_df = merged_df[merged_df['Season'].notna()]  # Drop rows with NaN in the "Season" column
merged_df = merged_df[merged_df['Season'] != '']    # Drop rows with empty strings in the "Season" column

# Step 2: Drop unnecessary columns
# Remove the "Season" and "Birthdate" columns as they are no longer required
merged_df = merged_df.drop(columns=['Season', 'Birthdate'])

# Step 3: Fill missing values in the "Competition" column
# Replace empty values in the "Competition" column with "International"
merged_df['Competition'] = merged_df['Competition'].fillna('International')

# Step 4: Filter out rows where "Games Played" has a value of "-"
# Only keep rows where "Games Played" contains valid data
merged_df = merged_df[merged_df['Games Played'] != '-']

# Save the cleaned and processed DataFrame to a new CSV file
output_path = '../data/cleaned/players_merged_stats_NT_cleaned.csv'
merged_df.to_csv(output_path, index=False)

# Display the first few rows of the cleaned DataFrame for verification
print(merged_df.head(10))

# Confirm that the processed file has been saved
print("Final data with 'Season' column processed and saved to:", output_path)


## 6. Cleaning Club Player Stats  

Clean club player statistics, add missing season information, and ensure uniform formatting for subsequent analysis.

In [ ]:
# Load the raw players club stats file
file_path = '../data/raw/players_club_stats.csv'
df = pd.read_csv(file_path)

# Check if the "Season" column exists in the DataFrame
if 'Season' in df.columns:
    # Step 1: Split the "Season" column by "-" to separate start and end years
    season_split = df['Season'].astype(str).str.split('-', expand=True)
    
    # Step 2: Create "Season Start" and "Season End" columns by processing the split values
    def convert_year(year):
        """
        Convert short year formats (e.g., 99 or 00) to full years (e.g., 1999 or 2000).
        """
        year = int(year)
        if 50 <= year <= 99:  # Assume years in the range 50-99 refer to the 1900s
            return 1900 + year
        elif 0 <= year <= 49:  # Assume years in the range 0-49 refer to the 2000s
            return 2000 + year
        return year  # Return the year unchanged if it's already in a valid format

    # Apply the conversion function to start and end years
    df['Season Start'] = season_split[0].apply(convert_year)
    # Use the start year if the end year is missing
    df['Season End'] = season_split[1].fillna(season_split[0]).apply(convert_year)

    # Step 3: Fill missing values in the "Competition" column with "International"
    df['Competition'] = df['Competition'].fillna('International')

    # Step 4: Save the cleaned DataFrame to a new CSV file
    output_path = '../data/cleaned/players_club_stats_cleaned.csv'
    df.to_csv(output_path, index=False)

    # Display the first few rows of the cleaned DataFrame for verification
    print(df.head(10))

    # Confirm where the cleaned file has been saved
    print("Final data with 'Competition' column filled saved to:", output_path)
else:
    # Handle the case where the "Season" column is not found in the DataFrame
    print("Error: 'Season' column not found in the DataFrame.")


## 7. Merging and Cleaning Files  

Merge club and national team statistics with player details to produce a consolidated dataset.

In [ ]:
# Function to clean and convert market values
def clean_market_value(value_str):
    """
    Standardizes and converts market values to a consistent format.
    Handles 'Mio.' (millions) and 'Tsd.' (thousands) formats.
    """
    value_str = value_str.replace('€', '').replace('â‚¬', '').replace('Tsd.', 'k').replace('Mio.', 'm').replace(',', '.').strip()
    market_value = 0.0
    if 'm' in value_str:  # If value is in millions
        market_value = float(value_str.replace('m', ''))
    elif 'k' in value_str:  # If value is in thousands
        market_value = float(value_str.replace('k', '')) / 1000  # Convert to millions
    return market_value

# Function to extract the birthdate from the Birthdate column
def extract_birthdate(birthdate_str):
    """
    Extracts the actual birthdate part from a string containing additional details in parentheses.
    """
    birthdate_part, _ = birthdate_str.split('(')
    return birthdate_part.strip()

# Function to parse a date string into a datetime object
def parse_date(date_str, format='%d.%m.%Y'):
    """
    Attempts to parse a date string into a datetime object using the provided format.
    Returns NaT if parsing fails.
    """
    try:
        return pd.to_datetime(date_str, format=format, errors='coerce')
    except ValueError:
        return pd.NaT

# Function to calculate age based on a birthdate
def calculate_age(birthdate):
    """
    Calculates the age of a player given their birthdate.
    """
    if pd.notnull(birthdate):
        today = datetime.today()
        # Calculate age by subtracting years and adjusting for months and days
        age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
        return age
    return None

# Path to the raw data file
file_path = '../data/raw/players_NT_2024-11-12.csv'  # Replace with the actual file path if different

# Load the data from the CSV file
df = pd.read_csv(file_path)

# Apply the function to clean and convert market values
df['Value'] = df['Value'].apply(clean_market_value)

# Extract and parse the birthdate into a consistent format
df['Birthdate'] = df['Birthdate'].apply(extract_birthdate).apply(lambda x: parse_date(x, format='%d.%m.%Y'))

# Calculate age based on the parsed birthdate
df['Age'] = df['Birthdate'].apply(calculate_age)

# Remove unnecessary spaces from the 'Name' column
df['Name'] = df['Name'].str.strip()

# (Optional) Parse other date columns such as 'ContractBegin' and 'ContractEnd'
# Uncomment the following lines if these columns exist in your data:
# df['ContractBegin'] = df['ContractBegin'].apply(lambda x: parse_date(x, format='%d.%m.%Y'))
# df['ContractEnd'] = df['ContractEnd'].apply(lambda x: parse_date(x, format='%d.%m.%Y'))

# Save the cleaned and modified DataFrame to a new CSV file
output_file_path = '../data/cleaned/players_NT_cleaned.csv'  # Replace with the desired output file name
df.to_csv(output_file_path, index=False)

# Display the first 10 rows of the cleaned DataFrame for verification
print(df.head(10))


In [ ]:
# Load cleaned data files
players_merged_stats_nt = pd.read_csv("../data/cleaned/players_merged_stats_NT_cleaned.csv")
players_detailed_stats_cleaned = pd.read_csv("../data/cleaned/players_club_stats_cleaned.csv")
players_incl_id = pd.read_csv("../data/cleaned/players_incl_ID.csv")
players_all_years = pd.read_csv("../data/cleaned/players_NT_cleaned.csv")

# Remove the 'insgesamt' column from the club stats DataFrame
players_detailed_stats_cleaned = players_detailed_stats_cleaned.drop(columns=['insgesamt'])

# Combine the national team stats and club stats DataFrames
combined_df = pd.concat([players_merged_stats_nt, players_detailed_stats_cleaned], ignore_index=True)

# Drop unnecessary columns 'National Team ID' and 'Position' if they exist
combined_df = combined_df.drop(columns=['National Team ID', 'Position'], errors='ignore')

# Add additional player information from players_incl_ID DataFrame
players_incl_id_selected = players_incl_id[['Player ID', 'Name', 'Position', 'Birthdate']]
combined_df = pd.merge(combined_df, players_incl_id_selected, on='Player ID', how='left')

# Add market value from players_all_years based on 'Name' and 'Season Start'
players_all_years_selected = players_all_years[['Name', 'Year', 'Value']]
final_df = pd.merge(
    combined_df, 
    players_all_years_selected, 
    how='left', 
    left_on=['Name', 'Season Start'], 
    right_on=['Name', 'Year']
)

# Remove the 'Year' column after merging
final_df = final_df.drop(columns=['Year'], errors='ignore')

# Reorganize columns into the desired order
final_df = final_df[
    [
        'Player ID', 'Name', 'Birthdate', 'Value', 'Competition', 'Type', 'Club', 'Debut Date', 
        'Season Start', 'Season End', 'Games Played', 'Goals', 'Assists', 'Own Goals', 
        'Substituted On', 'Substituted Off', 'Yellow Cards', 'Yellow Red', 'Red Cards', 
        'Penalty Goals', 'Minutes per Goal', 'Goals Conceded', 'Clean Sheets', 'Played Minutes'
    ]
]

# Display the first few rows of the final combined DataFrame for verification
print(final_df.head())

# Save the final DataFrame to a new CSV file
output_file_path = "../data/cleaned/players_combined.csv"
final_df.to_csv(output_file_path, index=False)

print(f"Combined DataFrame saved successfully to {output_file_path}")


 ## 8. Cleaning Columns

In [ ]:
# Load the combined player data
file_path = "../data/cleaned/players_combined.csv"
df = pd.read_csv(file_path)

# Function to clean and convert numeric columns
def clean_numeric_column(column):
    """
    Cleans a numeric column by removing unwanted characters such as quotes,
    replacing invalid values, and converting to float.
    """
    return df[column].astype(str).str.replace("'", "").str.replace(".", "").replace('-', '0').fillna('0').astype(float)

# Fill empty cells in the 'Value' column with 0 and scale values to millions
df['Value'] = df['Value'].fillna(0) * 1000000

# List of columns to clean by filling empty or invalid cells with 0 and converting to integers
columns_to_fill = [
    'Games Played', 'Goals', 'Assists', 'Own Goals', 'Substituted On', 'Substituted Off', 
    'Yellow Cards', 'Yellow Red', 'Red Cards', 'Penalty Goals', 'Minutes per Goal', 
    'Goals Conceded', 'Clean Sheets', 'Played Minutes'
]

# Apply the cleaning function to each column in the list and convert the results to integers
for column in columns_to_fill:
    df[column] = clean_numeric_column(column)
    df[column] = df[column].astype(int)

# Convert 'Debut Date' and 'Birthdate' columns to datetime format
df['Debut Date'] = pd.to_datetime(df['Debut Date'], format='%d/%m/%Y', errors='coerce')
df['Birthdate'] = pd.to_datetime(df['Birthdate'], format='%d/%m/%Y', errors='coerce')

# Check for null values in 'Debut Date' and 'Birthdate' after conversion
print("\nNull values after converting 'Debut Date' and 'Birthdate' to datetime:")
print(df[['Debut Date', 'Birthdate']].isnull().sum())

# Adjust 'Season Start' based on the 'Debut Date' month
# If the month is January to July, set 'Season Start' to the previous year
# If the month is August to December, set 'Season Start' to the year of 'Debut Date'
df['Season Start'] = df.apply(
    lambda row: row['Debut Date'].year - 1 if pd.notnull(row['Debut Date']) and pd.isnull(row['Season Start']) and row['Debut Date'].month <= 7 else 
                (row['Debut Date'].year if pd.notnull(row['Debut Date']) and pd.isnull(row['Season Start']) and row['Debut Date'].month >= 8 else row['Season Start']),
    axis=1
)

# Fill 'Season End' as 'Season Start' + 1 if 'Season End' is missing
df['Season End'] = df.apply(
    lambda row: row['Season Start'] + 1 if pd.notnull(row['Season Start']) and pd.isnull(row['Season End']) else row['Season End'],
    axis=1
)

# Verify the correctness of 'Season Start' and 'Season End' columns
print("\nChecking 'Season Start' and 'Season End' after filling logic:")
print(df[['Player ID', 'Debut Date', 'Season Start', 'Season End']].isnull().sum())

# Calculate 'Age in Season' using the 'Birthdate' and 'Season Start'
df['Birth Year'] = df['Birthdate'].dt.year
df['Age in Season'] = df['Season Start'] - df['Birth Year']

# Check for missing values in 'Age in Season'
print("\nMissing 'Age in Season' values:")
missing_age_rows = df[df['Age in Season'].isnull()]
print(missing_age_rows[['Player ID', 'Birthdate', 'Season Start', 'Age in Season']])

# Remove the helper column 'Birth Year' after calculation
df = df.drop(columns=['Birth Year'])

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = "../data/cleaned/players_combined_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Confirm that the cleaned data has been saved
print(f"\nCleaned data saved to: {cleaned_file_path}")


## 9. Grouping Leagues

In [ ]:

# Load the cleaned player dataset
file_path = "../data/cleaned/players_combined_cleaned.csv"
df = pd.read_csv(file_path)

# Fill empty cells in the 'Type' column with the default value 'Nationalteam'
df['Type'] = df['Type'].fillna('Nationalteam')

# Verify that all missing values in 'Type' are filled
print("Number of missing values in 'Type' after filling:", df['Type'].isnull().sum())

# Replace values in the 'Competition' column for "1. Liga CH"
df['Competition'] = df['Competition'].replace({
    '1. Liga Gr. 1': '1. Liga CH',
    '1. Liga Gr. 2': '1. Liga CH',
    '1. Liga Gr. 3': '1. Liga CH',
    '1. Liga Gr. 1 (- 11/12)': '1. Liga CH',
    '1 . Liga Gr. 2 (- 11/12)': '1. Liga CH',
    '1. Liga Gr. 3 (- 11/12)': '1. Liga CH',
    '1. Liga Playoff': '1. Liga CH'
})

# Replace values in the 'Competition' column for "2. Liga Inter CH"
df['Competition'] = df['Competition'].replace({
    '2. Liga Inter - Gr. 1': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 2': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 3': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 4': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 5': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 6': '2. Liga Inter CH',
    '2. Liga inter - Gr. 1 (- 11/12)': '2. Liga Inter CH',
    '2. Liga inter - Gr. 3 (- 11/12)': '2. Liga Inter CH',
    '2. Liga inter - Gr. 4 (- 11/12)': '2. Liga Inter CH',
    '2. Liga inter - Gr. 5 (- 11/12)': '2. Liga Inter CH',
    '2. Liga Inter - Gr. 6 (- 21/22)': '2. Liga Inter CH',
    '2. Liga': '2. Liga Inter CH'
})

# Verify that replacements in 'Competition' were successful
print("Unique values in 'Competition' after replacements:")
print(df['Competition'].unique())

# Save the updated DataFrame to a new CSV file
output_path = "../data/cleaned/players_combined_cleaned_updated.csv"
df.to_csv(output_path, index=False)

# Confirm the updated file has been saved
print(f"The file has been successfully updated and saved to {output_path}.")


## 10. Cleaning and Merging Market Values  

Standardize market values, fill missing data, and calculate player ages for consistent comparisons.

In [ ]:
# Define the current date for dynamic file path
current_date = datetime.now().strftime('%Y-%m-%d')
input_path = f'../data/raw/market_values_{current_date}.csv'
players_path = '../data/cleaned/players_incl_ID.csv'
output_path = f'../data/cleaned/market_values_{current_date}_cleaned.csv'

# Load the market values data
df = pd.read_csv(input_path)

# Load the players data to include "Birthdate"
players_df = pd.read_csv(players_path)

# Standardize column names for merging
players_df.rename(columns={'Player ID': 'Player_ID'}, inplace=True)

# Merge the market values data with players' birthdate data using 'Player_ID'
df = df.merge(players_df[['Player_ID', 'Birthdate']], on='Player_ID', how='left')

# Function to standardize the birthdate format to 'dd.mm.yyyy'
def standardize_birthdate(birthdate):
    try:
        return pd.to_datetime(birthdate, dayfirst=True).strftime('%d.%m.%Y')
    except Exception:
        return np.nan

# Apply the date conversion function for the 'Birthdate' column
df['Birthdate'] = df['Birthdate'].apply(standardize_birthdate)

# Function to handle various formats in the 'Market_Value' column and normalize the values
def convert_market_value(value):
    if pd.isna(value):
        return np.nan  # Keep NaN values as is

    # If the value is already numeric, return it
    if isinstance(value, (int, float)):
        return value

    # Clean and standardize string values
    value = value.strip().replace("â‚¬", "").replace("€", "").replace("Aktueller Marktwert:", "").strip()

    # Convert "-" to 0
    if value == "-":
        return 0

    # Extract numeric value from strings with "Mio." or "Tsd." and convert accordingly
    try:
        if "Mio" in value:
            number_str = re.search(r"(\d+[,.]?\d*)", value).group()
            number = float(number_str.replace(",", ".")) * 1_000_000
            return number
        elif "Tsd" in value:
            number_str = re.search(r"(\d+[,.]?\d*)", value).group()
            number = float(number_str.replace(",", ".")) * 1_000
            return number
        else:
            # Assume it's in euros if no unit is specified
            number_str = re.search(r"(\d+[,.]?\d*)", value).group()
            number = float(number_str.replace(",", "."))
            return number
    except (AttributeError, ValueError):
        return np.nan  # Return NaN if no valid number is found

# Apply the function to clean the 'Market_Value' column
df['Market_Value'] = df['Market_Value'].apply(convert_market_value)

# Function to standardize the 'Date' column to 'dd.mm.yyyy' format
def standardize_date(date):
    try:
        return pd.to_datetime(date, dayfirst=True).strftime('%d.%m.%Y')
    except Exception:
        return np.nan

# Apply the date standardization function
df['Date'] = df['Date'].apply(standardize_date)

# Function to calculate age if 'Age' is missing but 'Date' and 'Birthdate' are available
def calculate_age(birthdate, current_date):
    if pd.isna(birthdate) or pd.isna(current_date):
        return np.nan
    birthdate = pd.to_datetime(birthdate, dayfirst=True)
    current_date = pd.to_datetime(current_date, dayfirst=True)
    return current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))

# Apply age calculation for missing 'Age' values
df['Age'] = df.apply(lambda row: row['Age'] if not pd.isna(row['Age']) else calculate_age(row['Birthdate'], row['Date']), axis=1)

# Set 'Market_Value' to 0 where both 'Date' and 'Age' are present, but 'Market_Value' is missing
df.loc[(~df['Date'].isna()) & (~df['Age'].isna()) & (df['Market_Value'].isna()), 'Market_Value'] = 0

# Drop rows where 'Date' or 'Market_Value' are still missing
df = df.dropna(subset=['Date', 'Market_Value'])

# Select only relevant columns for the final output
df = df[['Player_ID', 'Date', 'Age', 'Market_Value', 'Birthdate']]

# Display the processed DataFrame for verification
print("\nFinal DataFrame with standardized Date, Market_Value, Birthdate, and calculated Age:")
print(df.head())

# Save the cleaned DataFrame to the specified output path
df.to_csv(output_path, index=False)

# Confirm the final output file has been saved
print(f"\nFinal cleaned data saved to {output_path}")


### Extending Market Values for all Ages

In [ ]:
# Define the current date and file paths
current_date = datetime.now().strftime('%Y-%m-%d')
market_values_path = f'../data/raw/market_values_{current_date}_cleaned.csv'
players_path = '../data/cleaned/players_incl_ID.csv'
output_path = f'../data/cleaned/market_values_{current_date}_extended.csv'

# Load the market values data and players data
market_values_df = pd.read_csv(market_values_path)
players_df = pd.read_csv(players_path)

# Rename 'Player ID' to 'Player_ID' in players_df to match the market_values_df column
players_df.rename(columns={'Player ID': 'Player_ID'}, inplace=True)

# Print columns to verify the alignment of data for merging
print("Columns in players_df:", players_df.columns)
print("Columns in market_values_df:", market_values_df.columns)

# Convert the 'Date' column in the market values dataset to datetime format for calculations
market_values_df['Date'] = pd.to_datetime(market_values_df['Date'], dayfirst=True)

# Identify the latest date in the market values data for reference
latest_date = market_values_df['Date'].max()
latest_date_str = latest_date.strftime('%d.%m.%Y')

# Convert 'Birthdate' in the players dataset to datetime format
players_df['Birthdate'] = pd.to_datetime(players_df['Birthdate'], dayfirst=True)

# Calculate the maximum age of each player as of the latest date in the market values data
players_df['Max_Age'] = players_df['Birthdate'].apply(
    lambda x: latest_date.year - x.year - ((latest_date.month, latest_date.day) < (x.month, x.day))
)

# Merge players' data with market values data to ensure all players are included
# Using suffixes to distinguish columns during the merge
full_df = pd.merge(
    players_df[['Player_ID', 'Birthdate', 'Max_Age']],
    market_values_df,
    on='Player_ID',
    how='left',
    suffixes=('_player', '_market')
)

# Use the 'Birthdate_player' column as the primary birthdate and drop redundant columns
full_df['Birthdate'] = full_df['Birthdate_player']
full_df = full_df.drop(columns=['Birthdate_player', 'Birthdate_market'], errors='ignore')

# Ensure 'Birthdate' exists in the final DataFrame, raise an error if missing
if 'Birthdate' not in full_df.columns:
    print("Columns after processing:", full_df.columns)
    raise KeyError("The 'Birthdate' column is missing from the final DataFrame.")

# Define a function to fill missing ages by generating rows for the entire age range
def fill_missing_ages(group):
    # Generate a DataFrame for the full age range (14 to max age for the player)
    max_age = group['Max_Age'].iloc[0]
    full_age_range = pd.DataFrame({'Age': range(14, max_age + 1)})
    
    # Merge the age range with the player's existing data
    merged_df = full_age_range.merge(group, on='Age', how='left')
    
    # Fill missing fields with appropriate values
    merged_df['Player_ID'] = group['Player_ID'].iloc[0]
    merged_df['Birthdate'] = group['Birthdate'].iloc[0]
    merged_df['Date'] = merged_df['Date'].fillna(latest_date_str)
    merged_df['Market_Value'] = merged_df['Market_Value'].fillna(0)  # Set missing market values to 0
    
    return merged_df

# Apply the function to generate rows for missing ages for each player
filled_df = full_df.groupby('Player_ID').apply(fill_missing_ages).reset_index(drop=True)

# Select only the relevant columns for the final output
filled_df = filled_df[['Player_ID', 'Date', 'Age', 'Market_Value', 'Birthdate']]

# Display the resulting DataFrame for verification
print("\nFinal DataFrame with filled age rows and Market_Value set to 0 where missing:")
print(filled_df.head())  # Adjust to view more rows if needed

# Save the final DataFrame to the output file
filled_df.to_csv(output_path, index=False)

# Confirm the data has been saved
print(f"\nFilled data saved to {output_path}")


### Completing market values based on preceding values

In [ ]:
# Define the current date and file paths
current_date = datetime.now().strftime('%Y-%m-%d')  # Format the current date dynamically
market_values_path = f'../data/cleaned/market_values_2024-11-15_cleaned.csv'  # Path to the cleaned market values file
players_path = '../data/cleaned/players_incl_ID.csv'  # Path to the players' data file
output_path = f'../data/cleaned/market_values_{current_date}_extended.csv'  # Output path for the extended market values file

# Load the market values data and players data
market_values_df = pd.read_csv(market_values_path)  # Load the market values file
players_df = pd.read_csv(players_path)  # Load the players file

# Rename 'Player ID' to 'Player_ID' in players_df to match the column name in market_values_df
players_df.rename(columns={'Player ID': 'Player_ID'}, inplace=True)

# Print the columns in both DataFrames to ensure alignment before merging
print("Columns in players_df:", players_df.columns)
print("Columns in market_values_df:", market_values_df.columns)

# Convert the 'Date' column in the market values data to datetime format
market_values_df['Date'] = pd.to_datetime(market_values_df['Date'], dayfirst=True)

# Identify the latest date in the market values data for reference
latest_date = market_values_df['Date'].max()
latest_date_str = latest_date.strftime('%d.%m.%Y')  # Format the latest date as a string in 'dd.mm.yyyy'

# Convert the 'Birthdate' column in the players data to datetime format
players_df['Birthdate'] = pd.to_datetime(players_df['Birthdate'], dayfirst=True)

# Calculate the maximum age of each player as of the latest date
players_df['Max_Age'] = players_df['Birthdate'].apply(
    lambda x: latest_date.year - x.year - ((latest_date.month, latest_date.day) < (x.month, x.day))
)

# Merge the players' data with the market values data, ensuring all players are included
full_df = pd.merge(
    players_df[['Player_ID', 'Birthdate', 'Max_Age']],  # Only include necessary columns from players_df
    market_values_df,  # Merge with market values data
    on='Player_ID',  # Merge on Player_ID
    how='left',  # Include all players, even if no market value data is available
    suffixes=('_player', '_market')  # Distinguish overlapping columns with suffixes
)

# Use 'Birthdate_player' as the primary birthdate column and drop redundant columns
full_df['Birthdate'] = full_df['Birthdate_player']
full_df = full_df.drop(columns=['Birthdate_player', 'Birthdate_market'], errors='ignore')

# Ensure that the 'Birthdate' column exists in the final DataFrame
if 'Birthdate' not in full_df.columns:
    print("Columns after processing:", full_df.columns)
    raise KeyError("The 'Birthdate' column is missing from the final DataFrame.")

# Define a function to fill missing ages for each player
def fill_missing_ages(group):
    # Generate the full range of ages from 14 to the player's maximum age
    max_age = group['Max_Age'].iloc[0]
    full_age_range = pd.DataFrame({'Age': range(14, max_age + 1)})
    
    # Merge the full age range with the player's existing data
    merged_df = full_age_range.merge(group, on='Age', how='left')
    
    # Fill missing 'Player_ID' and 'Birthdate' values
    merged_df['Player_ID'] = group['Player_ID'].iloc[0]
    merged_df['Birthdate'] = group['Birthdate'].iloc[0]
    merged_df['Date'] = merged_df['Date'].fillna(latest_date_str)  # Fill missing dates with the latest date
    
    # Fill missing Market_Value with the last recorded value (forward fill)
    merged_df['Market_Value'] = merged_df['Market_Value'].fillna(method='ffill')
    
    # Set any remaining missing Market_Value to 0
    merged_df['Market_Value'] = merged_df['Market_Value'].fillna(0)
    
    return merged_df

# Apply the function to each player's data, ensuring all ages from 14 to max age are included
filled_df = full_df.groupby('Player_ID').apply(fill_missing_ages).reset_index(drop=True)

# Retain only relevant columns for the final output
filled_df = filled_df[['Player_ID', 'Date', 'Age', 'Market_Value', 'Birthdate']]

# Display the resulting DataFrame for verification
print("\nFinal DataFrame with filled age rows and Market_Value updated:")
print(filled_df.head())  # Preview the first few rows of the DataFrame

# Save the extended DataFrame to a new output file
filled_df.to_csv(output_path, index=False)

# Confirm the data has been saved successfully
print(f"\nFilled data saved to {output_path}")


### Combine Datasets

In [ ]:
# File paths
players_file = "../data/cleaned/players_combined_cleaned_updated.csv"  # Path to the cleaned players data
market_values_file = "../data/cleaned/market_values_2024-11-18_extended.csv"  # Path to the extended market values data
player_incl_id_file = "../data/cleaned/players_incl_ID.csv"  # Path to the player details file

# Step 1: Load the market values data
market_values_df = pd.read_csv(market_values_file)

# Step 2: Drop 'Date' and 'Birthdate' columns
# These columns are not required for the merging process and analysis
market_values_df = market_values_df.drop(columns=['Date', 'Birthdate'], errors='ignore')

# Step 3: Group by 'Player_ID' and 'Age' to calculate the average market value for each age
market_values_avg = market_values_df.groupby(['Player_ID', 'Age'])['Market_Value'].mean().reset_index()

# Step 4: Create a helper column for the last recorded value of 'Market_Value' for each player and age
# This is done using forward fill within each player group to retain historical market value data
market_values_avg = market_values_avg.sort_values(by=['Player_ID', 'Age'])
market_values_avg['Last_Recorded_Value'] = market_values_avg.groupby('Player_ID')['Market_Value'].ffill()

# Step 5: Load the cleaned players data and remove duplicate rows
players_df = pd.read_csv(players_file)
players_df.drop_duplicates(inplace=True)  # Ensure no duplicate rows exist in the data

# Step 6: Merge the players data with the market values
# Perform a left join on 'Player ID' and 'Age in Season' to align the market values with player data
merged_df = pd.merge(
    players_df,
    market_values_avg,
    how='left',
    left_on=['Player ID', 'Age in Season'],
    right_on=['Player_ID', 'Age']
)

# Step 7: Handle missing values in 'Market_Value'
# Use 'Last_Recorded_Value' to fill missing values in 'Market_Value'
merged_df['Market_Value'] = merged_df['Market_Value'].combine_first(merged_df['Last_Recorded_Value'])
merged_df['Market_Value'] = merged_df['Market_Value'].fillna(0)  # Replace any remaining missing values with 0

# Drop unnecessary columns that are no longer required for analysis
merged_df = merged_df.drop(columns=['Age', 'Player_ID', 'Last_Recorded_Value', 'Value'], errors='ignore')

# Step 8: Load the player details file to include the 'Position' column
player_incl_id_df = pd.read_csv(player_incl_id_file)

# Step 9: Merge the 'Position' column into the merged DataFrame
# Use 'Player ID' as the key to bring position data into the final DataFrame
merged_with_position = pd.merge(
    merged_df,
    player_incl_id_df[['Player ID', 'Position']],
    how='left',
    on='Player ID'
)

# Step 10: Save the updated DataFrame to a new CSV file
output_file = "../data/cleaned/players_complete.csv"  # Path to save the final DataFrame
merged_with_position.to_csv(output_file, index=False)

# Print confirmation of successful saving
print(f"Final processed data with positions saved to {output_file}")


## 11. Separate National Team Levels

In [ ]:
# File path
players_complete_file = "../data/cleaned/players_complete.csv"  # Path to the cleaned players dataset

# Step 1: Load the players dataset
players_df = pd.read_csv(players_complete_file)  # Load the players data for further processing

# Step 2: Define a function to determine the 'Category' based on 'Type' and 'Club'
import re  # Import the regex module for pattern matching

def determine_category(row):
    """
    Determines the category of a player based on their 'Type' and 'Club'.

    If the 'Type' is 'Nationalteam':
    - Extract the U-level (e.g., 'U15', 'U17') from the 'Club' using regex if present.
    - If no U-level is found, categorize as 'A Nationalteam'.
    For non-national teams, the category remains None.

    Args:
        row: A row of the DataFrame.

    Returns:
        str: The category of the player.
    """
    if row['Type'] == 'Nationalteam':  # Check if the player belongs to the national team
        if 'U' in row['Club']:  # Check if the club contains a U-level (e.g., U15, U21)
            # Use regex to extract "U" followed by digits (e.g., 'U15', 'U17')
            match = re.search(r'U\d+', row['Club'])
            if match:
                return match.group(0)  # Return the matched U-level
        else:
            return 'A Nationalteam'  # Default to 'A Nationalteam' for senior players
    return None  # Return None for non-national team players

# Step 3: Apply the function to create a new 'Category' column
# Apply the determine_category function to each row of the DataFrame
players_df['Category'] = players_df.apply(determine_category, axis=1)

# Step 4: Save the updated DataFrame to a new CSV file
output_file = "../data/cleaned/players_complete.csv"  # Path to save the updated dataset
players_df.to_csv(output_file, index=False)  # Save the DataFrame without including the index

# Print confirmation message
print(f"Updated data with fixed 'Category' column saved to {output_file}")


## 12. Add Top 5 Competition Yes/No

In [ ]:
# First clean Bundesliga Germany / Bundesliga AUT

import pandas as pd

# File path
players_complete_file = "../data/cleaned/players_complete.csv"  # Path to the cleaned players dataset

# Load the players_complete.csv file
players_df = pd.read_csv(players_complete_file)  # Load the players data for further processing

# List of Bundesliga AUT clubs
# A predefined list of Austrian Bundesliga clubs to differentiate from the German Bundesliga
bundesliga_aut_clubs = [
    "FC Blau-Weiß Linz", "FC Wacker Innsbruck", "FK Austria Wien", 
    "LASK", "SCR Altach", "SK Rapid Wien", "SK Sturm Graz", 
    "Red Bull Salzburg", "SV Ried"
]

# Step 1: Define a function to clean the "Competition" column
def clean_competition(row):
    """
    Adjusts the 'Competition' column to distinguish between German Bundesliga and Austrian Bundesliga.
    
    Args:
        row: A row of the DataFrame.
        
    Returns:
        str: Updated competition name.
    """
    if row['Competition'] == "Bundesliga" and row['Club'] in bundesliga_aut_clubs:
        return "Bundesliga AUT"  # Mark as Austrian Bundesliga
    return row['Competition']  # Retain original competition otherwise

# Step 2: Apply the function to update the "Competition" column
players_df['Competition'] = players_df.apply(clean_competition, axis=1)

# Step 4: Define a list of top 5 competitions
# The top-tier European leagues to be identified in the dataset
top_5_competitions = ["Serie A", "Bundesliga", "Ligue 1", "Premier League", "LaLiga"]

# Step 5: Define a function to determine if the competition is in the Top 5
def is_top_5(competition):
    """
    Determines if the competition is one of the top 5 leagues.
    
    Args:
        competition (str): The competition name.
        
    Returns:
        str: "Yes" if the competition is in the top 5, otherwise "No".
    """
    return "Yes" if competition in top_5_competitions else "No"

# Step 6: Apply the function to create a new 'Top 5' column
players_df['Top 5'] = players_df['Competition'].apply(is_top_5)

# Step 8: Define a function to determine if the player is in the A Nationalteam
def is_a_nationalteam(category):
    """
    Identifies if the player is part of the A Nationalteam.
    
    Args:
        category (str): The player's category (e.g., "A Nationalteam").
        
    Returns:
        str: "Yes" if in A Nationalteam, otherwise "No".
    """
    return "Yes" if category == "A Nationalteam" else "No"

# Step 9: Apply the function to create a new 'A Nationalteam' column
players_df['A Nationalteam'] = players_df['Category'].apply(is_a_nationalteam)

# Step 11: Define a function to determine the general position
def determine_general_position(position):
    """
    Maps specific positions to generalized categories (e.g., Goalkeeper, Defender).
    
    Args:
        position (str): The player's specific position.
        
    Returns:
        str: The generalized position category.
    """
    if position == "Torwart":
        return "Goalkeeper"
    elif position in ["Linker Verteidiger", "Innenverteidiger", "Rechter Verteidiger", "Abwehr"]:
        return "Defender"
    elif position in ["Defensives Mittelfeld", "Linkes Mittelfeld", "Rechtes Mittelfeld", "Zentrales Mittelfeld", "Offensives Mittelfeld", "Mittelfeld"]:
        return "Midfielder"
    elif position in ["Hängende Spitze", "Linksaußen", "Rechtsaußen", "Mittelstürmer", "Sturm"]:
        return "Forward"
    return "Unknown"  # Default category for unexpected positions

# Step 12: Apply the function to create the 'General Position' column
players_df['General Position'] = players_df['Position'].apply(determine_general_position)

# Step 3: Save the updated DataFrame back to a CSV file
output_file = "../data/cleaned/players_complete.csv"  # Path to save the updated dataset
players_df.to_csv(output_file, index=False)  # Save the DataFrame without including the index

# Print confirmation message
print(f"Updated data with cleaned 'Competition' column saved to {output_file}")


## 13. Grouping into Swiss / Foreign

In [ ]:
import pandas as pd

# Load the dataset
file_path = '../data/cleaned/players_complete.csv'  # Path to the cleaned players dataset
df = pd.read_csv(file_path)

# Define a list of competitions that belong to Swiss football leagues
swiss_competitions = [
    "1. Liga CH", "Super League", "Challenge League", "Challenge League Barrage", "2. Liga Inter CH",
    "U17 Elite League", "U17 Elite Platzierungsspiele", "U17 Elite Playoffs", "U17 Elite Playouts",
    "U18 Elite League", "U18 Elite Platzierungsspiele", "U18 Elite Playoffs", "U18 Elite Playouts",
    "U19 Elite League", "U19 Elite Platzierungsspiele", "U19 Elite Playoffs", "U19 Elite Playouts",
    "U21 PL Elite Gruppe",
    "Promotion League", "Promotion League - Abstiegsr.", "Promotion League - Aufstiegsr.", "Promotion League - Quali",
    "Super League Barrage", "Super League Play-off", "Super League Play-out"
]

# Step 1: Create a new column "Country"
def assign_country(competition):
    """
    Assigns the country based on the competition.
    
    Args:
        competition (str): The competition name.
    
    Returns:
        str: The assigned country ("Swiss", "Foreign", or "N/A").
    """
    if competition in swiss_competitions:
        return "Swiss"
    elif competition == "International":
        return "N/A"
    else:
        return "Foreign"

df['Country'] = df['Competition'].apply(assign_country)

# Step 2: Create a new column "Level"
def assign_level(row):
    """
    Assigns the level (e.g., Academy, Active, or N/A) based on the club and type.
    
    Args:
        row: A row of the DataFrame.
    
    Returns:
        str: The assigned level.
    """
    if row['Type'] == 'Nationalteam':
        return "N/A"
    elif any(f"U{age}" in row['Club'] for age in range(15, 24)):
        return "Academy"
    else:
        return "Active"

df['Level'] = df.apply(assign_level, axis=1)

# Step 3: Identify clubs that participated in Swiss competitions
clubs_in_swiss_competitions = df[df['Competition'].isin(swiss_competitions)]['Club'].unique()

# Update "Country" for rows where the club is in the identified Swiss clubs list
df['Country'] = df.apply(
    lambda row: "Swiss" if row['Club'] in clubs_in_swiss_competitions else row['Country'], axis=1
)

# Step 4: Combine "Country" and "Level" into a new column "Country_Level"
def combine_country_and_level(row):
    """
    Combines the 'Country' and 'Level' columns into a single column.
    
    Args:
        row: A row of the DataFrame.
    
    Returns:
        str: The combined value of country and level, or "N/A" if either is "N/A".
    """
    if row['Country'] == "N/A" or row['Level'] == "N/A":
        return "N/A"
    else:
        return f"{row['Country']} - {row['Level']}"

df['Country_Level'] = df.apply(combine_country_and_level, axis=1)

# Save the updated DataFrame to a new CSV file
output_file = '../data/cleaned/players_complete_updated.csv'  # Output path for the updated dataset
df.to_csv(output_file, index=False)  # Save without including the index

# Confirm the dataset has been saved
print(f"Filtered dataset with 'Country' and 'Country_Level' columns saved to {output_file}")


## 14. Get Similar Players 

In [ ]:
# Reload data
file_path = '../data/cleaned/players_completed.csv'  # Load the updated cleaned players dataset
players_df = pd.read_csv(file_path)

# Define numeric features for similarity calculations
features = [
    'Games Played', 'Goals', 'Assists', 'Yellow Cards', 'Red Cards',
    'Penalty Goals', 'Minutes per Goal', 'Goals Conceded', 'Clean Sheets',
    'Played Minutes'
]
players_df[features] = players_df[features].fillna(0)  # Fill NaN values with 0 for numerical stability

# Encode categorical features: 'Competition', 'General Position', and 'Country_Level'
encoder = OneHotEncoder(sparse_output=False)  # Use OneHotEncoder to convert categorical data to binary
encoded = encoder.fit_transform(players_df[['Competition', 'General Position', 'Country_Level']])
encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(['Competition', 'General Position', 'Country_Level'])  # Add meaningful column names
)

# Convert 'Top 5' column to binary (1 for 'Yes', 0 for 'No')
players_df['Top 5'] = players_df['Top 5'].map({'Yes': 1, 'No': 0}).fillna(0)

# Combine encoded categorical features back into the DataFrame
players_df = pd.concat([players_df.reset_index(drop=True), encoded_df], axis=1)

# Filter baseline players belonging to the 'A Nationalteam'
baseline_players = players_df[players_df['Category'] == 'A Nationalteam']

# Filter rows where 'Age in Season' is below 22 for both baseline and all players
baseline_players = baseline_players[baseline_players['Age in Season'] < 22]
players_df = players_df[players_df['Age in Season'] < 22]

# Calculate total occurrences of "Top 5" competitions per player per Age in Season
top_5_totals = (
    players_df.groupby(['Player ID', 'Age in Season'])['Top 5']
    .sum()
    .reset_index()
    .rename(columns={'Top 5': 'Top 5 Total'})  # Rename for clarity
)

# Aggregate Top 5 totals across all ages as a feature for each player
top_5_aggregated = (
    top_5_totals.groupby('Player ID')['Top 5 Total']
    .sum()
    .reset_index()
    .rename(columns={'Top 5 Total': 'Top 5 Aggregated'})
)

# Merge aggregated "Top 5" totals back into the main DataFrame
players_df = players_df.merge(top_5_aggregated, on='Player ID', how='left')
baseline_players = baseline_players.merge(top_5_aggregated, on='Player ID', how='left')

# Update the list of features to include encoded columns and 'Top 5 Aggregated'
encoded_features = encoded_df.columns.tolist()
all_features = features + encoded_features + ['Top 5 Aggregated']

# Aggregate cumulative stats for each player by summing all rows
players_aggregated = players_df.groupby(['Player ID', 'Name'])[all_features].sum().reset_index()
baseline_aggregated = baseline_players.groupby(['Player ID', 'Name'])[all_features].sum().reset_index()

# Check if data for comparison exists
if players_aggregated.empty or baseline_aggregated.empty:
    print("No players or baseline players left after filtering. Exiting...")
else:
    # Step 1: Scale features for similarity calculation
    scaler = StandardScaler()
    scaler.fit(baseline_aggregated[all_features])

    # Apply scaling to player stats
    players_scaled = scaler.transform(players_aggregated[all_features])
    baseline_scaled = scaler.transform(baseline_aggregated[all_features])

    # Step 2: Compute similarity between each player and baseline players
    similar_results_all = []
    for idx, player in players_aggregated.iterrows():
        player_id, player_name = player['Player ID'], player['Name']  # Extract player info

        # Scale the player's features
        player_scaled = players_scaled[idx].reshape(1, -1)

        # Calculate cosine similarity with baseline players
        similarities = cosine_similarity(player_scaled, baseline_scaled).flatten()

        # Combine similarity scores with baseline player details
        baseline_aggregated_copy = baseline_aggregated.copy()
        baseline_aggregated_copy['Similarity'] = similarities

        # Exclude the player itself from the similarity results
        baseline_aggregated_copy = baseline_aggregated_copy[baseline_aggregated_copy['Player ID'] != player_id]

        # Identify the top 3 most similar players
        top_3 = (
            baseline_aggregated_copy.sort_values(by='Similarity', ascending=False)
            .head(3)
        )

        # Record the similarity results
        similar_results_all.append({
            'Player ID': player_id,
            'Player Name': player_name,
            'First Similar Player': top_3.iloc[0]['Name'] if len(top_3) > 0 else None,
            'Similarity Score 1': top_3.iloc[0]['Similarity'] if len(top_3) > 0 else None,
            'Second Similar Player': top_3.iloc[1]['Name'] if len(top_3) > 1 else None,
            'Similarity Score 2': top_3.iloc[1]['Similarity'] if len(top_3) > 1 else None,
            'Third Similar Player': top_3.iloc[2]['Name'] if len(top_3) > 2 else None,
            'Similarity Score 3': top_3.iloc[2]['Similarity'] if len(top_3) > 2 else None
        })

    # Step 3: Save the similarity results
    all_results_df = pd.DataFrame(similar_results_all)
    output_file_all = '../data/cleaned/all_similar_players.csv'  # Path for saving results
    all_results_df.to_csv(output_file_all, index=False)

    # Confirm saving the results
    print(f"All results saved to {output_file_all}")
